In [2]:
from random import randint
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import io
import requests
from time import sleep, time


merge_result_stocks=pd.DataFrame()


confirmed_cases_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
deaths_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"

read_confirmed_Cases = pd.read_csv(confirmed_cases_url)
df = pd.DataFrame(read_confirmed_Cases)


read_dead_Cases = pd.read_csv(deaths_url)
df1 = pd.DataFrame(read_dead_Cases)



confirmed_cases_melt = pd.melt(
    df, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'])
confirmed_cases_melt['variable'] = pd.to_datetime(
    confirmed_cases_melt['variable'], format='%m/%d/%y')
confirmed_cases_melt['variable'] = confirmed_cases_melt['variable'].astype(
    'datetime64[ns]')
confirmed_cases_melt = confirmed_cases_melt.rename(
    columns={'variable': 'date', 'value': 'confirmedCases'})

confirmed_cases_melt = confirmed_cases_melt.groupby('date', as_index=False)[
                                                     'confirmedCases'].sum()


dead_case_melt = pd.melt(
    df1, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'])
dead_case_melt['variable'] = pd.to_datetime(
    dead_case_melt['variable'], format='%m/%d/%y')
dead_case_melt['variable'] = dead_case_melt['variable'].astype(
    'datetime64[ns]')
dead_case_melt = dead_case_melt.rename(
    columns={'variable': 'date', 'value': 'DeathCases'})

dead_case_melt = dead_case_melt.groupby('date', as_index=False)[
                                         'DeathCases'].sum()


merge_data_sources = pd.merge(confirmed_cases_melt, dead_case_melt, on=['date'])
print(merge_data_sources)

f = open('creds.json')

data = json.load(f)

apiKey = data['apiKey']
travelStock = data['travelStock']
realEstateStock = data['realEstateStock']
metalStock = data['metalStock']
americanStock = data['americanStock']
canadaStock = data['canadaStock']


stocklists = []

stocklists.append(americanStock)
stocklists.append(canadaStock)
stocklists.append(travelStock)
stocklists.append(realEstateStock)
stocklists.append(metalStock)

URL = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&outputsize=full&apikey="+apiKey+"&datatype=csv"

def API(URL,stock):
    api = URL+"&symbol="+stock
    urldata = requests.get(api).content
    stock_df = pd.DataFrame(pd.read_csv(io.StringIO(urldata.decode('utf-8'))))
    if('timestamp' in stock_df):          
          stock_df = stock_df.drop(columns=['open', 'volume'])
          stock_df = stock_df.rename(columns={'timestamp': 'date'})
          stock_df = stock_df.rename(columns={'high': stock+'_high'})
          stock_df = stock_df.rename(columns={'low': stock+'_low'})
          stock_df = stock_df.rename(columns={'close': stock+'_close'})
          stock_df['date'] = stock_df['date'].astype('datetime64')
          stock_df[stock] = stock
          return stock_df     


df1 = pd.concat([API(URL,x) for x in stocklists],sort=True).sort_index()
merge_result_StockData = pd.merge(merge_data_sources, df1, on=['date'])
stockdata_df=merge_result_StockData.fillna("N/A")
print(stockdata_df)


       
      

ModuleNotFoundError: No module named 'pandas'

In [ ]:
stockdata_df.to_csv('StockData.csv',index=False)

In [6]:
df = pd.read_csv('StockData.csv')
df = df[['confirmedCases','DeathCases','AAPL,'AAPL_close','AAPL_high','AAPL_low','AC','AC_close','AC_high', 'AC_low','AMT','AMT_close','AMT_high','AMT_low','NUE','NUE_close','NUE_high','NUE_low','SHOP','SHOP_close','SHOP_high','SHOP_low']]
df.hist(figsize=(18,15),bins=31, linewidth='4',edgecolor='blue',grid=False)
plt.show('confirmedCases','DeathCases')

SyntaxError: invalid syntax (1626392292.py, line 2)

In [ ]:
df2 = df2[['AAPL_close','AAPL_high','AAPL_low']]
df1.hist(figsize=(18,15),bins=31, linewidth='4',edgecolor='blue',grid=False)
plt.show('AAPL_close','AAPL_high','AAPL_low')